In [9]:
import pandas as pd
import numpy as np

# 1. Load kedua dataset
df_dataset1 = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terdahulu_grouping.csv')
df_dataset2 = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini_dengan_label.csv')

# 2. Sesuaikan nama kolom agar konsisten
# Dataset 1 (berita_terdahulu_grouping.csv) sudah memiliki kolom yang sesuai
# Hanya perlu rename Created_at ke date
df_dataset1 = df_dataset1.rename(columns={'Created_at': 'date'})

# Dataset 2 (berita_terkini_dengan_label.csv) perlu disesuaikan:
# - title -> Title
# - content -> Content  
# - predicted_label -> label
# - date tetap date
df_dataset2 = df_dataset2.rename(columns={
    'title': 'Title',
    'content': 'Content',
    'predicted_label': 'label'
})

# 3. Pilih hanya kolom yang dibutuhkan untuk kedua dataset
required_columns = ['Title', 'Content', 'stemmed_text', 'faktor_str', 'label', 'date']

df_dataset1_selected = df_dataset1[required_columns]
df_dataset2_selected = df_dataset2[required_columns]

# 4. Hapus duplikat dan nilai kosong dari kedua dataset
# df_dataset1_clean = df_dataset1_selected.drop_duplicates(subset=['Content'])
# df_dataset1_clean = df_dataset1_clean.dropna(subset=['Content', 'Title'])

# df_dataset2_clean = df_dataset2_selected.drop_duplicates(subset=['Content'])
# df_dataset2_clean = df_dataset2_clean.dropna(subset=['Content', 'Title'])

# 5. Gabungkan kedua dataset (tanpa sampling)
df_combined = pd.concat([df_dataset1_selected, df_dataset2_selected], ignore_index=True)

# 6. Acak dataset gabungan untuk memastikan data tercampur dengan baik
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# 7. Simpan dataset gabungan ke file CSV
df_combined.to_csv('berita_combined.csv', index=False)

# 8. Tampilkan informasi dataset
print(f"Dataset 1 (berita_terdahulu_grouping):")
print(f"  - Data asli: {len(df_dataset1)}")
print(f"  - Setelah cleaning: {len(df_dataset1_selected)}")

print(f"\nDataset 2 (berita_terkini_dengan_label):")
print(f"  - Data asli: {len(df_dataset2)}")
print(f"  - Setelah cleaning: {len(df_dataset2_selected)}")

print(f"\nDataset gabungan:")
print(f"  - Total data: {len(df_combined)}")

print(f"\nKolom-kolom dalam dataset gabungan:")
print(df_combined.columns.tolist())

print(f"\nInfo dataset gabungan:")
print(df_combined.info())

print(f"\nContoh 5 baris pertama dari dataset gabungan:")
print(df_combined.head())

Dataset 1 (berita_terdahulu_grouping):
  - Data asli: 1212
  - Setelah cleaning: 1212

Dataset 2 (berita_terkini_dengan_label):
  - Data asli: 1078
  - Setelah cleaning: 1078

Dataset gabungan:
  - Total data: 2290

Kolom-kolom dalam dataset gabungan:
['Title', 'Content', 'stemmed_text', 'faktor_str', 'label', 'date']

Info dataset gabungan:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         2290 non-null   object
 1   Content       2290 non-null   object
 2   stemmed_text  2290 non-null   object
 3   faktor_str    2290 non-null   object
 4   label         2290 non-null   object
 5   date          2290 non-null   object
dtypes: object(6)
memory usage: 107.5+ KB
None

Contoh 5 baris pertama dari dataset gabungan:
                                               Title  \
0  Aliansi Buruh-Tani Ancam Mogok Massal Jika Per...   
1  Terpo

In [10]:
# Analisis distribusi label dari dataset gabungan
import pandas as pd

# Load dataset gabungan yang sudah dibuat
df_combined = pd.read_csv('berita_combined.csv')

print("=== ANALISIS DISTRIBUSI LABEL ===")

# 1. Distribusi label secara keseluruhan
print(f"\nDataset Gabungan - Distribusi Label:")
label_dist = df_combined['label'].value_counts().sort_index()
print(label_dist)
print(f"Total data: {label_dist.sum()}")

# 2. Persentase distribusi label
print(f"\nDataset Gabungan - Persentase Label:")
label_percentage = df_combined['label'].value_counts(normalize=True).sort_index() * 100
for label, percentage in label_percentage.items():
    count = label_dist[label]
    print(f"  {label}: {count} data ({percentage:.2f}%)")

# 3. Cek apakah ada nilai null di kolom label
null_labels = df_combined['label'].isnull().sum()
print(f"\nJumlah data dengan label kosong: {null_labels}")

# 4. Unique labels yang ada
print(f"\nLabel unik yang tersedia:")
unique_labels = df_combined['label'].unique()
for label in sorted(unique_labels):
    if pd.notna(label):  # Skip jika ada NaN
        print(f"  - {label}")

# 5. Visualisasi sederhana
print(f"\nVisualisasi Distribusi:")
max_count = label_dist.max()
for label in sorted(label_dist.index):
    count = label_dist[label]
    bar_length = int((count / max_count) * 50)  # Scale to 50 characters max
    bar = "█" * bar_length
    print(f"{label:>8}: {bar} ({count})")

=== ANALISIS DISTRIBUSI LABEL ===

Dataset Gabungan - Distribusi Label:
negatif    1542
positif     748
Name: label, dtype: int64
Total data: 2290

Dataset Gabungan - Persentase Label:
  negatif: 1542 data (67.34%)
  positif: 748 data (32.66%)

Jumlah data dengan label kosong: 0

Label unik yang tersedia:
  - negatif
  - positif

Visualisasi Distribusi:
 negatif: ██████████████████████████████████████████████████ (1542)
 positif: ████████████████████████ (748)


In [12]:
import pandas as pd
from datetime import datetime
import re

# Load dataset gabungan
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_combined.csv')

# Mapping nama bulan Indonesia ke angka
BULAN_INDONESIA = {
    'jan': 1, 'januari': 1,
    'feb': 2, 'februari': 2,
    'mar': 3, 'maret': 3,
    'apr': 4, 'april': 4,
    'mei': 5, 'may': 5,
    'jun': 6, 'juni': 6,
    'jul': 7, 'juli': 7,
    'agu': 8, 'agustus': 8,
    'sep': 9, 'september': 9,
    'okt': 10, 'oktober': 10,
    'nov': 11, 'november': 11,
    'des': 12, 'desember': 12
}

# Mapping nama hari Indonesia (opsional, untuk validasi)
HARI_INDONESIA = {
    'senin': 'monday',
    'selasa': 'tuesday', 
    'rabu': 'wednesday',
    'kamis': 'thursday',
    'jumat': 'friday',
    'sabtu': 'saturday',
    'minggu': 'sunday'
}

def standardize_date_complete(date_str):
    """
    Mengkonversi berbagai format tanggal termasuk format Indonesia menjadi dd/mm/yyyy
    """
    if pd.isna(date_str) or date_str == '':
        return None
    
    # Convert to string dan bersihkan
    date_str = str(date_str).strip()
    original_date_str = date_str
    
    # Hapus timezone indicator dan WIB
    date_str = re.sub(r'\+\d{2}$', '', date_str)
    date_str = re.sub(r'UTC$', '', date_str)
    date_str = re.sub(r'WIB$', '', date_str)
    date_str = date_str.strip()
    
    # Penanganan khusus untuk format Indonesia: "Jumat, 20 Des 2024 17:01"
    indonesia_pattern = r'^(\w+),?\s+(\d{1,2})\s+(\w+)\s+(\d{4})(?:\s+\d{1,2}:\d{1,2}(?::\d{1,2})?)?$'
    indonesia_match = re.match(indonesia_pattern, date_str)
    
    if indonesia_match:
        try:
            hari, tanggal, bulan_str, tahun = indonesia_match.groups()
            bulan_str = bulan_str.lower()
            
            if bulan_str in BULAN_INDONESIA:
                bulan = BULAN_INDONESIA[bulan_str]
                parsed_date = datetime(int(tahun), bulan, int(tanggal))
                return parsed_date.strftime('%d/%m/%Y')
        except ValueError as e:
            print(f"Error parsing Indonesian date '{original_date_str}': {e}")
    
    # List format tanggal standar
    date_formats = [
        # Format dengan timezone sudah dihapus
        '%Y-%m-%d %H:%M:%S',    # 2023-03-13 09:18:44
        '%Y-%m-%d %H:%M',       # 2023-03-13 09:18
        '%Y-%m-%d',             # 2023-03-13
        '%Y/%m/%d %H:%M:%S',    # 2023/03/13 09:18:44
        '%Y/%m/%d %H:%M',       # 2023/03/13 09:18
        '%Y/%m/%d',             # 2023/03/13
        
        # Format d/m/yyyy
        '%d/%m/%Y %H:%M:%S',    # 13/03/2023 09:18:44
        '%d/%m/%Y %H:%M',       # 13/03/2023 09:18
        '%d/%m/%Y',             # 13/03/2023
        '%d-%m-%Y %H:%M:%S',    # 13-03-2023 09:18:44
        '%d-%m-%Y %H:%M',       # 13-03-2023 09:18
        '%d-%m-%Y',             # 13-03-2023
        
        # Format m/d/yyyy (US format)
        '%m/%d/%Y %H:%M:%S',    # 03/13/2023 09:18:44
        '%m/%d/%Y %H:%M',       # 03/13/2023 09:18
        '%m/%d/%Y',             # 03/13/2023
        
        # Format tanpa separator
        '%Y%m%d',               # 20230313
        
        # Format dengan nama bulan Inggris
        '%d-%b-%Y',             # 13-Mar-2023
        '%d %B %Y',             # 13 March 2023
        '%B %d, %Y',            # March 13, 2023
        '%d %b %Y',             # 13 Mar 2023
    ]
    
    # Coba parsing dengan berbagai format
    for fmt in date_formats:
        try:
            parsed_date = datetime.strptime(date_str, fmt)
            return parsed_date.strftime('%d/%m/%Y')
        except ValueError:
            continue
    
    # Penanganan khusus untuk format dengan angka tanpa leading zero
    patterns = [
        # yyyy-m-d atau yyyy/m/d
        (r'^(\d{4})[-/](\d{1,2})[-/](\d{1,2})(?:\s+\d{1,2}:\d{1,2}(?::\d{1,2})?)?$', 'yyyy-mm-dd'),
        # m/d/yyyy dengan atau tanpa waktu (US format)
        (r'^(\d{1,2})/(\d{1,2})/(\d{4})(?:\s+\d{1,2}:\d{1,2}(?::\d{1,2})?)?$', 'mm/dd/yyyy'),
        # d-m-yyyy atau d/m/yyyy
        (r'^(\d{1,2})[-/](\d{1,2})[-/](\d{4})(?:\s+\d{1,2}:\d{1,2}(?::\d{1,2})?)?$', 'dd-mm-yyyy'),
    ]
    
    for pattern, format_type in patterns:
        match = re.match(pattern, date_str)
        if match:
            groups = match.groups()
            try:
                if format_type == 'yyyy-mm-dd':
                    year, month, day = groups
                elif format_type == 'mm/dd/yyyy':
                    month, day, year = groups
                else:  # dd-mm-yyyy
                    day, month, year = groups
                
                # Validasi tanggal
                parsed_date = datetime(int(year), int(month), int(day))
                return parsed_date.strftime('%d/%m/%Y')
            except ValueError:
                # Jika gagal, mungkin format US (m/d/yyyy) salah interpretasi
                if format_type == 'dd-mm-yyyy':
                    try:
                        # Coba sebagai US format (m/d/yyyy)
                        month, day, year = groups
                        parsed_date = datetime(int(year), int(month), int(day))
                        return parsed_date.strftime('%d/%m/%Y')
                    except ValueError:
                        continue
                continue
    
    # Jika semua gagal, return nilai asli
    print(f"Warning: Could not parse date '{original_date_str}', keeping original value")
    return original_date_str

# Backup data asli jika belum ada
if 'date_original' not in df.columns:
    print("Membuat backup kolom date asli...")
    df['date_original'] = df['date'].copy()

# Test beberapa contoh format Indonesia
print("=== TEST FORMAT INDONESIA ===")
test_dates = [
    "Jumat, 20 Des 2024 17:01 WIB",
    "Rabu, 26 Jun 2024 19:17 WIB", 
    "Senin, 13 Mei 2024 19:57 WIB"
]

print("Test konversi format Indonesia:")
for test_date in test_dates:
    result = standardize_date_complete(test_date)
    print(f"  {test_date} → {result}")

# Konversi format tanggal dengan fungsi lengkap
print("\n=== MELAKUKAN KONVERSI DENGAN DUKUNGAN FORMAT INDONESIA ===")
print("Mengkonversi ke format dd/mm/yyyy...")

df['date'] = df['date_original'].apply(standardize_date_complete)

# Cek hasil konversi
print("\n=== HASIL KONVERSI ===")
print("Sample tanggal setelah konversi:")
sample_dates_new = df['date'].dropna().head(15).tolist()
for i, date in enumerate(sample_dates_new, 1):
    print(f"{i:2d}. {date}")

# Validasi hasil konversi
print(f"\nTotal data dengan tanggal setelah konversi: {df['date'].notna().sum()}")
print(f"Total data tanpa tanggal setelah konversi: {df['date'].isna().sum()}")

# Cek apakah ada tanggal yang tidak terkonversi dengan benar
non_standard_dates = []
successfully_converted = 0

for date in df['date'].dropna():
    if re.match(r'^\d{2}/\d{2}/\d{4}$', str(date)):
        successfully_converted += 1
    else:
        non_standard_dates.append(date)

# Hitung tingkat keberhasilan konversi
total_dates = len(df['date'].dropna())
success_rate = (successfully_converted / total_dates * 100) if total_dates > 0 else 0

print(f"\nTingkat keberhasilan konversi: {success_rate:.1f}% ({successfully_converted}/{total_dates})")

if non_standard_dates:
    print(f"\nPeringatan: {len(non_standard_dates)} tanggal masih tidak sesuai format dd/mm/yyyy:")
    # Ambil sample unik untuk analisis
    unique_non_standard = list(set(non_standard_dates))[:10]
    for date in unique_non_standard:
        print(f"  - {date}")
    if len(unique_non_standard) > 10:
        print(f"  ... dan {len(unique_non_standard) - 10} format lainnya")
else:
    print(f"\n✅ Semua tanggal berhasil dikonversi ke format dd/mm/yyyy")

# Simpan dataset dengan tanggal yang sudah distandarisasi
df.to_csv('berita_combined_standardized_date.csv', index=False)
print(f"\n✅ Dataset dengan tanggal terstandarisasi disimpan ke: 'berita_combined_standardized_date.csv'")

# Tampilkan contoh perbandingan
print(f"\n=== SAMPLE PERBANDINGAN SEBELUM DAN SESUDAH ===")
comparison_sample = df[['date_original', 'date']].head(20)
for idx, row in comparison_sample.iterrows():
    original = str(row['date_original'])
    converted = str(row['date'])
    status = "✅" if re.match(r'^\d{2}/\d{2}/\d{4}$', converted) else "❌"
    # Potong teks jika terlalu panjang
    original_short = original[:40] + "..." if len(original) > 40 else original
    converted_short = converted[:40] + "..." if len(converted) > 40 else converted
    print(f"{status} {original_short:<43} → {converted_short}")

# Analisis format bermasalah yang tersisa
if non_standard_dates:
    print(f"\n=== ANALISIS FORMAT YANG MASIH BERMASALAH ===")
    problem_categories = {
        'Format Indonesia belum tertangani': 0,
        'Format dengan timezone kompleks': 0,
        'Format ambiguous': 0,
        'Format tidak dikenal': 0
    }
    
    sample_problems = unique_non_standard[:20]  # Ambil sample untuk analisis
    for date_str in sample_problems:
        date_str = str(date_str)
        if any(bulan in date_str.lower() for bulan in BULAN_INDONESIA.keys()):
            problem_categories['Format Indonesia belum tertangani'] += 1
        elif '+' in date_str or 'GMT' in date_str or 'UTC' in date_str:
            problem_categories['Format dengan timezone kompleks'] += 1
        elif '/' in date_str and len(date_str.split('/')) == 3:
            problem_categories['Format ambiguous'] += 1
        else:
            problem_categories['Format tidak dikenal'] += 1
    
    print("Kategori masalah yang tersisa:")
    for category, count in problem_categories.items():
        if count > 0:
            print(f"  - {category}: {count} kasus")
            
# Informasi tambahan tentang bulan Indonesia yang didukung
print(f"\n=== FORMAT INDONESIA YANG DIDUKUNG ===")
print("Bulan yang dapat dikonversi:")
bulan_list = list(BULAN_INDONESIA.keys())
for i in range(0, len(bulan_list), 6):
    print(f"  {', '.join(bulan_list[i:i+6])}")

print(f"\nContoh format yang didukung:")
print(f"  - Jumat, 20 Des 2024 17:01 WIB")
print(f"  - Senin, 15 Jan 2024 10:30")
print(f"  - Rabu, 05 Februari 2024")

Membuat backup kolom date asli...
=== TEST FORMAT INDONESIA ===
Test konversi format Indonesia:
  Jumat, 20 Des 2024 17:01 WIB → 20/12/2024
  Rabu, 26 Jun 2024 19:17 WIB → 26/06/2024
  Senin, 13 Mei 2024 19:57 WIB → 13/05/2024

=== MELAKUKAN KONVERSI DENGAN DUKUNGAN FORMAT INDONESIA ===
Mengkonversi ke format dd/mm/yyyy...

=== HASIL KONVERSI ===
Sample tanggal setelah konversi:
 1. 15/03/2023
 2. 15/03/2023
 3. 30/08/2024
 4. 02/05/2023
 5. 27/03/2023
 6. 06/07/2024
 7. 28/03/2023
 8. 21/03/2023
 9. 06/04/2023
10. 10/12/2022
11. 21/11/2024
12. 09/01/2024
13. 20/12/2024
14. 17/11/2023
15. 07/12/2023

Total data dengan tanggal setelah konversi: 2288
Total data tanpa tanggal setelah konversi: 2

Tingkat keberhasilan konversi: 100.0% (2288/2288)

✅ Semua tanggal berhasil dikonversi ke format dd/mm/yyyy

✅ Dataset dengan tanggal terstandarisasi disimpan ke: 'berita_combined_standardized_date.csv'

=== SAMPLE PERBANDINGAN SEBELUM DAN SESUDAH ===
✅ 2023-03-15 00:32:32+00                     

In [13]:
import pandas as pd
from datetime import datetime
import re

# Load dataset yang sudah distandarisasi tanggalnya
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_combined_standardized_date.csv')

print(f"=== SORTING DATA BERDASARKAN TANGGAL (2020-2024) ===")
print(f"Total data awal: {len(df)}")

# Fungsi untuk convert tanggal dd/mm/yyyy ke datetime
def parse_date_to_datetime(date_str):
    """
    Convert format dd/mm/yyyy ke datetime object
    """
    if pd.isna(date_str) or date_str == '':
        return None
    
    date_str = str(date_str).strip()
    
    # Cek apakah sudah format dd/mm/yyyy
    if re.match(r'^\d{2}/\d{2}/\d{4}$', date_str):
        try:
            return datetime.strptime(date_str, '%d/%m/%Y')
        except ValueError:
            return None
    
    # Jika belum format standar, coba parse format lain
    formats_to_try = [
        '%Y-%m-%d',           # 2024-01-15
        '%Y/%m/%d',           # 2024/01/15
        '%d-%m-%Y',           # 15-01-2024
        '%m/%d/%Y',           # 01/15/2024
        '%Y-%m-%d %H:%M:%S',  # 2024-01-15 10:30:45
        '%d/%m/%Y %H:%M:%S',  # 15/01/2024 10:30:45
    ]
    
    for fmt in formats_to_try:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    return None

# Convert kolom date ke datetime
print("Mengkonversi tanggal ke format datetime...")
df['date_datetime'] = df['date'].apply(parse_date_to_datetime)

# Cek berapa yang berhasil dikonversi
converted_count = df['date_datetime'].notna().sum()
print(f"Berhasil mengkonversi {converted_count} dari {len(df)} tanggal ke datetime")

# Filter data untuk tahun 2020-2024
print("\nMemfilter data untuk tahun 2020-2024...")
df_with_date = df[df['date_datetime'].notna()].copy()

# Filter berdasarkan tahun
df_filtered = df_with_date[
    (df_with_date['date_datetime'].dt.year >= 2020) & 
    (df_with_date['date_datetime'].dt.year <= 2024)
].copy()

print(f"Data setelah filter tahun 2020-2024: {len(df_filtered)}")

# Analisis distribusi per tahun sebelum sorting
print(f"\n=== DISTRIBUSI DATA PER TAHUN ===")
year_distribution = df_filtered['date_datetime'].dt.year.value_counts().sort_index()
for year, count in year_distribution.items():
    print(f"  {year}: {count} data")

# Sort berdasarkan tanggal dari terbaru ke lama (descending)
print(f"\nMengurutkan data dari tanggal terbaru ke lama...")
df_sorted = df_filtered.sort_values('date_datetime', ascending=False).reset_index(drop=True)

# Hapus kolom helper datetime jika tidak diperlukan
df_final = df_sorted.drop('date_datetime', axis=1)

print(f"\n=== HASIL SORTING ===")
print(f"Total data final: {len(df_final)}")

# Tampilkan sample 10 data teratas (terbaru)
print(f"\n10 Data Terbaru:")
sample_newest = df_final[['Title', 'date']].head(10)
for idx, row in sample_newest.iterrows():
    title_short = str(row['Title'])[:50] + "..." if len(str(row['Title'])) > 50 else str(row['Title'])
    print(f"  {idx+1:2d}. [{row['date']}] {title_short}")

# Tampilkan sample 10 data terlama
print(f"\n10 Data Terlama:")
sample_oldest = df_final[['Title', 'date']].tail(10)
for idx, row in sample_oldest.iterrows():
    title_short = str(row['Title'])[:50] + "..." if len(str(row['Title'])) > 50 else str(row['Title'])
    relative_idx = len(df_final) - (len(df_final) - idx - 1)
    print(f"  {relative_idx:2d}. [{row['date']}] {title_short}")

# Cek tanggal terbaru dan terlama
if len(df_sorted) > 0:
    newest_date = df_sorted['date_datetime'].iloc[0]
    oldest_date = df_sorted['date_datetime'].iloc[-1]
    print(f"\nRentang waktu data:")
    print(f"  Terbaru: {newest_date.strftime('%d/%m/%Y')} ({newest_date.strftime('%A, %d %B %Y')})")
    print(f"  Terlama: {oldest_date.strftime('%d/%m/%Y')} ({oldest_date.strftime('%A, %d %B %Y')})")

# Analisis distribusi per bulan tahun terbaru
if len(df_sorted) > 0:
    latest_year = df_sorted['date_datetime'].iloc[0].year
    print(f"\n=== DISTRIBUSI BULAN TAHUN {latest_year} ===")
    latest_year_data = df_sorted[df_sorted['date_datetime'].dt.year == latest_year]
    if len(latest_year_data) > 0:
        month_dist = latest_year_data['date_datetime'].dt.month.value_counts().sort_index()
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'Mei', 'Jun', 
                      'Jul', 'Agu', 'Sep', 'Okt', 'Nov', 'Des']
        for month, count in month_dist.items():
            print(f"  {month_names[month-1]} {latest_year}: {count} data")

# Simpan data yang sudah diurutkan
output_filename = 'berita_sorted_2020_2024.csv'
df_final.to_csv(output_filename, index=False)
print(f"\n✅ Data yang sudah diurutkan berdasarkan tanggal disimpan ke: '{output_filename}'")

# Validasi akhir
print(f"\n=== VALIDASI AKHIR ===")
print(f"✅ Data diurutkan dari: {year_distribution.index.max()} hingga {year_distribution.index.min()}")
print(f"✅ Total data: {len(df_final)}")
print(f"✅ Kolom yang tersimpan: {list(df_final.columns)}")

# Cek apakah ada duplikat berdasarkan content
duplicates = df_final.duplicated(subset=['Content']).sum()
if duplicates > 0:
    print(f"⚠️  Terdeteksi {duplicates} data duplikat berdasarkan content")
    print(f"   Jika ingin menghapus duplikat, jalankan: df_final.drop_duplicates(subset=['Content'], inplace=True)")
else:
    print(f"✅ Tidak ada duplikat berdasarkan content")

=== SORTING DATA BERDASARKAN TANGGAL (2020-2024) ===
Total data awal: 2290
Mengkonversi tanggal ke format datetime...
Berhasil mengkonversi 2288 dari 2290 tanggal ke datetime

Memfilter data untuk tahun 2020-2024...
Data setelah filter tahun 2020-2024: 2279

=== DISTRIBUSI DATA PER TAHUN ===
  2020: 29 data
  2021: 19 data
  2022: 68 data
  2023: 970 data
  2024: 1193 data

Mengurutkan data dari tanggal terbaru ke lama...

=== HASIL SORTING ===
Total data final: 2279

10 Data Terbaru:
   1. [31/12/2024] Prabowo Ungkap Alasan Umumkan Sendiri Kenaikan PPN...
   2. [31/12/2024] Harga BBM di Tahun 2025 Bakal Turun? Begini Perkir...
   3. [31/12/2024] Geger Jutaan Masyarakat Kelas Menengah RI Turun Ka...
   4. [31/12/2024] Drama Rupiah: Ketika Jokowi Ketar-ketir Gegara Dol...
   5. [31/12/2024] Respons Maman soal Dampak Kebijakan Proteksionisme...
   6. [31/12/2024] 10 Negara dengan Harga Rumah Paling Mahal di Dunia...
   7. [31/12/2024] Waka DPR sebut Kenaikan PPN 12% Amanat UU, Tak Aka...

In [14]:
df_final.drop_duplicates(subset=['Content'], inplace=True)